Notebook implementation of the self-ask + Google tool use prompt.
Adapted from https://github.com/ofirpress/self-ask

In [1]:
from dataclasses import dataclass
from parsita import *
import minichain

Define the state of the bot.

In [2]:
@dataclass
class IntermediateState:
    s: str

In [3]:
@dataclass
class FinalState:
    s: str

In [4]:
@dataclass
class Out:
    echo: str
    state: FinalState | IntermediateState

Self Ask Prompt

In [5]:
class SelfAsk(minichain.TemplatePrompt[Out]):
    template_file = "selfask.pmpt.tpl"
    stop_template = "\nIntermediate answer:"

    # Parsita parser.
    class Parser(TextParsers):
        follow = (lit("Follow up:") >> reg(r".*")) > IntermediateState
        finish = (lit("So the final answer is: ") >> reg(r".*")) > FinalState
        response = follow | finish

    def parse(self, response: str, inp) -> Out:
        return Out(
            self.prompt(inp).prompt + response,
            self.Parser.response.parse(response).or_die(),
        )

Runtime loop

In [6]:
def selfask(inp: str, openai, google) -> str:
    prompt1 = SelfAsk(openai)
    prompt2 = minichain.SimplePrompt(google)
    suffix = ""
    for i in range(3):
        out = prompt1(dict(input=inp, suffix=suffix, agent_scratchpad=True))

        if isinstance(out.state, FinalState):
            break
        suffix += out.echo
        out2 = prompt2(out.state.s)
        suffix += "\nIntermediate answer: " + out2 + "\n"
    return out.state.s

In [7]:
with minichain.start_chain("selfask") as backend:
    result = selfask(
        "What is the zip code of the city where George Washington was born?",
        backend.OpenAI(),
        backend.Google(),
    )
    print(result)

https://serpapi.com/search


https://serpapi.com/search


22443, 22520, 22469, 22488, 22442, 22524, 22529, 22558


View prompt examples.

In [8]:
SelfAsk().show(
    {
        "input": "What is the zip code of the city where George Washington was born?",
        "agent_scratchpad": True,
    },
    "Follow up: Where was George Washington born?",
)

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>SelfAsk</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="p">{</span><span class="s1">&#39;input&#39;</span><span class="p">:</span> <span class="s1">&#39;What is the zip code of the city where George Washington was born?&#39;</span><span class="p">,</span> <span class="s1">&#39;agent_scratchpad&#39;</span><span class="p">:</span> <span class="kc">True</span><span class="p">}</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary>Prompt</summary>\n      <p>Question: Who lived longer, Muhammad Ali or Alan Turing?<br>Are follow up questions needed here: Yes.<br>Follow up: How old was Muhammad Ali when he died?<br>Intermediate answer: Muhammad Ali was 74 years old when he died.<br>Follow up: How old was Alan Turing when he died?<br>Intermediate answer: Alan Turing was 41 years old when he died.<br>So the final answer is: Muhammad Ali<br><br>Question: When was the founder of craigslist born?<br>Are follow up questions needed here: Yes.<br>Follow up: Who was the founder of craigslist?<br>Intermediate answer: Craigslist was founded by Craig Newmark.<br>Follow up: When was Craig Newmark born?<br>Intermediate answer: Craig Newmark was born on December 6, 1952.<br>So the final answer is: December 6, 1952<br><br>Question: Who was the maternal grandfather of George Washington?<br>Are follow up questions needed here: Yes.<br>Follow up: Who was the mother of George Washington?<br>Intermediate answer: The mother of George Washington was Mary Ball Washington.<br>Follow up: Who was the father of Mary Ball Washington?<br>Intermediate answer: The father of Mary Ball Washington was Joseph Ball.<br>So the final answer is: Joseph Ball<br><br>Question: Are both the directors of Jaws and Casino Royale from the same country?<br>Are follow up questions needed here: Yes.<br>Follow up: Who is the director of Jaws?<br>Intermediate answer: The director of Jaws is Steven Spielberg.<br>Follow up: Where is Steven Spielberg from?<br>Intermediate answer: The United States.<br>Follow up: Who is the director of Casino Royale?<br>Intermediate answer: The director of Casino Royale is Martin Campbell.<br>Follow up: Where is Martin Campbell from?<br>Intermediate answer: New Zealand.<br>So the final answer is: No<br><br>Question: <div style=\'color:red\'>What is the zip code of the city where George Washington was born?</div><br>Are followup questions needed here: Yes.<br></p>\n    </details>\n  </dd>\n\n  <dt> Response: </dt>\n  <dd>\n    Follow up: Where was George Washington born?\n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">Out</span><span class="p">(</span><span class="n">echo</span><span class="o">=</span><span class="s1">&#39;Question: Who lived longer, Muhammad Ali or Alan Turing?</span><span class="se">\\n</span><span class="s1">Are follow up questions needed here: Yes.</span><span class="se">\\n</span><span class="s1">Follow up: How old was Muhammad Ali when he died?</span><span class="se">\\n</span><span class="s1">Intermediate answer: Muhammad Ali was 74 years old when he died.</span><span class="se">\\n</span><span class="s1">Follow up: How old was Alan Turing when he died?</span><span class="se">\\n</span><span class="s1">Intermediate answer: Alan Turing was 41 years old when he died.</span><span class="se">\\n</span><span class="s1">So the final answer is: Muhammad Ali</span><span class="se">\\n\\n</span><span class="s1">Question: When was the founder of craigslist born?</span><span class="se">\\n</span><span class="s1">Are follow up questions needed here: Yes.</span><span class="se">\\n</span><span class="s1">Follow up: Who was the founder of craigslist?</span><span class="se">\\n</span><span class="s1">Intermediate answer: Craigslist was founded by

View log.

In [9]:
minichain.show_log("selfask.log")

a1394473-6f63-4eda-a13c-b04d79cc2ad4
└── <class '__main__.SelfAsk'>/1 ⇒ started 2023-02-27 14:18:10Z ⧖ 1.268s
    ├── Input Function/2/1 ⇒ started 2023-02-27 14:18:10Z ⧖ 0.004s
    │   ├── input: 
    │   │   ├── agent_scratchpad: True
    │   │   ├── input: What is the zip code of the city where George Washington was born?
    │   │   └── suffix: 
    │   └── Input Function/2/2 ⇒ succeeded 2023-02-27 14:18:10Z
    ├── Prompted/3/1 ⇒ started 2023-02-27 14:18:10Z ⧖ 1.258s
    │   ├── prompt: Question: Who lived longer, Muhammad Ali or Alan Turing?⏎
    │   │   Are follow up questions needed here: Yes.⏎
    │   │   Follow up: How old was Muhammad Ali when he died?⏎
    │   │   Intermediate answer: Muhammad Ali was 74 years old when he died.⏎
    │   │   Follow up: How old was Alan Turing when he died?⏎
    │   │   Intermediate answer: Alan Turing was 41 years old when he died.⏎
    │   │   So the final answer is: Muhammad Ali⏎
    │   │   ⏎
    │   │   Question: When was the founder of c